In [ ]:
import sys
sys.path.insert(0,  "C:\\Users\\Paul Valcke\\Documents\\GitHub\\CHIMES" )
import chimes as chm


from IPython.display import display,HTML,Markdown
from IPython.display import IFrame
from itables import init_notebook_mode,options
options.columnDefs = [{"className": "dt-left", "targets": "_all"}]
options.classes="display nowrap compact"
options.scrollY="400px"
options.scrollCollapse=True
options.paging=False

init_notebook_mode(all_interactive=True)
%matplotlib widget

# Pandas display
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option("display.colheader_justify","left")

## Tomas Mazak's workaround
import plotly
import plotly.graph_objs as go
from IPython.display import display, HTML
plotly.offline.init_notebook_mode()
display(HTML('<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'))

# TL:DR (Too long, didn't read)

Here is the resume of everything seen in this tutorial
```
# TO KNOW WHAT YOU CAN LOAD
chm.get_available_models() # to know model names 
chn.get_available_saves() # to know saves names

# TO LOAD A MODEL
hub=chm.Hub(              # CHIMES object that you can use to interact with a model
            'modelname',
            'presetname', # Optional, if you want to start with a specific set of values 
            verb=False,   # Optional, if you say true 
            )
hub=chm.load_saved(
            'savename or full adress'
)
hub.get_summary()           # Basic general informations
hub.get_fieldsproperties()  # dataframe description of fields
hub.get_Network()           # network visualisation of the model
hub.get_presets()           # display a table of the presets
hub.get_supplements()       # display a table of the supplements
hub.get_dfields()           # dictionnary of all fields properties
hub.dfields_as_reverse_dict(crit='eqtype') # classified by eqtypes
values = hub.get_dvalues(idx=0,         # By default idx=0
                         Region=0,      # By default region=0
                         params=False   # If True, return parameters too, otherwise only time evolving quantities
                         )

hub.dataframe()             # fields values as multiindex dataframe
hub.dflags                  # status of the model
hub.dfunc_order             # order of the functions resolution
hub.dmisc                   # miscelaneous information
hub.dmodel                  # What is read from the model file
```

# Exploring a model 

Models are made to interact with. All the interactions can be done through a CHIMES object called 'Hub`. Here, we show how to use it with one model, a Lorenz System (the famous chaos butterfly). 

In [ ]:
chm.get_available_models() # We check model names to be sure how we write it

In [ ]:
name = 'Lorenz_Attractor'
chm.get_model_documentation(name) # We look deeper to its properties

## Creating a hub



### Constructing a Hub

The normal approach is to construct a Hub around a model. It is a CHIMES object that will decypher the model file and give you methods to interact with it. 

When you initialize a Huh with a model, it will: 
* Find what are all the fields that exist in the model
* Determine if they are differential, state variable, parameters
* Try to find as many existing information as possible (typically in CHIMES library) that has not been put by the model creator but exist in a common database. It can be :
    * units 
    * symbol
    * definition 
    * default value
    * ...
* Find an order to calculate the system at each timestep, and what variables are necessary (in loops)
* Preparing allocation for future time calculation and make calculation faster
* Making the model elements ready to be displayed in a user-friendly manner
* A bit of coffee for you 

In this example we call the created object `hub` (although the name could be anything you like !)

In [ ]:
hub=chm.Hub(name) # Load the model file into a hub

### Calling an existing Hub

A Hub can be saved with its model and runs using `hub.save(name, description)`. 
They can be loaded in `hub=chm.load(name)`

## Exploring the content of the model

a `hub` has many introspection methods that return formatted data, that can be called as `hub.get_[...]`

### Summary
In most situations, the method `get_summary()` should cover your needs. 
It covers the flags of the system, theiur preset names, and fields by category with their attributes. As we will see after, get_summary displays the last value of the simulation when a run has been done.

In [ ]:
hub.get_summary()

### Field properties

A dataframe version is also possible using `hub.get_fieldproperties`, when using itables it give you the possibility to do search in the system. 

In [ ]:
hub.get_fieldsproperties()

### Field properties, as a network 

As we said, a model is a web of fields linked by equations. We can thus represent it as such. and interact with it. 
The `get_Network()` method allow the customization of the representation.

You can hover on each nodes to get more information about its properties and dependencies 

In [ ]:
hub.get_Network()

### See the presets and supplements 

Presets are a set of value for the system given by the author of a model, that allow one specific run and the display of a specific behavior of the system (either quantitative or qualitative). They are often associated with one serie of plots that shows it. They can be loaded into the hub using `set_preset(presetname)`. To see the presets, use `hub.get_preset`, or read `hub.get_summary()` or even `chm.get_model_documentation`

Supplements are functions that ease the use of a specific model and thus associated to it, either for the analyses, setting fields, or plots. they can be accessed the same way using `hub.get_supplements` or `get_model_documentation`. they can be then used with `hub.supplements[supplementname](arguments_of_the_function)`

In [ ]:
presets = hub.get_presets('list') # Return a list of the presets
dpresets = hub.get_presets('dict') # Return a dictionary of the presets with their description
hub.get_presets() # display a table of the presets

In [ ]:
presets = hub.get_supplements('list') # Return a list of the supplements
dpresets = hub.get_supplements('dict') # Return a dictionary of the supplements
hub.get_supplements() # display a table of the supplements

### dfields 

dfields is where all the fields properties of the model, digested by the hub, are located. It is a dictionnary of a dictionnary architecture: the first level of dfields is fields names, then their attributes, such as their value, definition and so on. 

the keys are: 
* 'eqtype' type of equation ['differential', 'statevar', 'None']
* 'func' the python logic that is used to calculate the value
* 'source_exp' A string version of the solved equation
* 'com' a comment explaining the python function
* 'definition' the definition of the field
* 'units' the unit of the field ( years, humans, meters...) in latex formalism
* 'symbol' a latex symbol that represent the field
* 'group' a classification that helps looking
* 'value' a numpy tensor that keep all values (see the corresponding section) 
* 'isneeded' Is the field required to calculate the dynamics
* 'size' a list of two field that are used to know the multisectoral size of this one
* 'multisect' a flag that tell if the field need all its dimensions
* 'kargs'  
* 'args'  
* 'minmax' 

`get_dfields` give you the possibility to filter which key you want to explore: 

In [42]:
R = hub.get_dfields()
print('the available fields are :',R.keys())
x = R['x']
print('the keys of the field x are :',x.keys())

the available fields are : dict_keys(['x', 'y', 'z', 'distance', 'lor_sigma', 'lor_rho', 'lor_beta', 'Tsim', 'Tini', 'dt', 'nx', 'nr', 'Nprod', '__ONE__', 'time', 'nt'])
the keys of the field x are : dict_keys(['func', 'com', 'definition', 'units', 'initial', 'eqtype', 'size', 'value', 'minmax', 'symbol', 'group', 'multisect', 'kargs', 'args', 'source_exp', 'isneeded'])


In [41]:
hub.get_dfields(eqtype=['differential']).keys() # You only get differential equations

dict_keys(['x', 'y', 'z', 'time'])

### Getting fields according to criterias

It can be usedfull to have a specfic subset of keys according to a certain classification. `get_dfields_as_reverse_dict` is doing so

In [40]:
hub.dfields_as_reverse_dict(   # Return a dictionnary with the list of outputs for each input
                            crit='units', # classified by units
                            eqtype=['differential', 'statevar']
                            ) # only eqtype differential and statevar are kept


{None: ['lor_sigma', 'lor_rho', 'lor_beta', 'Tsim', 'Tini', 'dt', '__ONE__'],
 'statevar': ['distance'],
 'size': ['nx', 'nr', 'Nprod'],
 'parameter': ['nt'],
 'differential': ['x', 'y', 'z', 'time']}

In [ ]:
hub.dfields_as_reverse_dict(crit='eqtype') # classified by eqtypes


# Getting values in most cases

If you do not need the multisectoral properties, you can get a dictionnary of values with `dvalue`. 
If you have multiple regions of parrallel system you can slice only the one you want with nx = index, region = index

In [ ]:
values = hub.get_dvalues(idx=0,         # By default idx=0
                         Region=0,      # By default region=0
                         params=False   # If True, return parameters too, otherwise only time evolving quantities
                         )


### Dataframe approach

You can also access the values as a dataframe, that reshapes the dfields structure.

In [ ]:
df = hub.dataframe().transpose()

## Decorators 

Those varaiables are not shaped for user readability but they can still be accessed. Those are protected and thus cannot be modified directly

In [ ]:
hub.dflags # status of the model

In [ ]:
hub.dfunc_order # order of the functions resolution

In [ ]:
hub.dmisc # miscelaneous information

In [ ]:
hub.dmodel # What is read from the model file